# 2. Zynq 芯片
***
Zynq 的总的架构包含了两个部分：处理器系统 （PS）和可编程逻辑 （PL）。这两部分可以单独使用，也可以合起来用，而且实际上供电电路被设计成独立给每个部分供电，这样 PS 或 PL 部分不被使用的话就可以被断电。不过，Zynq 最有价值的模式是它的两个组成部分结合起来使用，因此理解两个部分的结构以及两者之间的接口是很重要的。
![](zynq_struct.png)
本章接下去的部分就是从 PS 部分开始来分析 Zynq 的架构。


## 2.1 处理器系统
所有的 Zynq 芯片都有相同的基本架构。作为处理器系统的基础，所有的芯片都一颗“硬”处理器 —— 它是芯片上专用而且优化过的硅片元件。

“软” 处理器，是由可编程逻辑部分的单元组合而成的。也就是说，一个软处理器的实现和部署在 FPGA 的逻辑结构里的任何其他 IP 包是等价的。一般来说，软处理器的优势是处理器实例的数量和精确实现是灵活的。从另一方面来说，硬处理器可以获得相对较高的性能。
***
可以在 Zynq 的 **PL 部分**配上一个或多个**软处理器**，用来和硬处理器**协同工作**。  
比如这些 MicroBlaze 处理器可以负责协调特定的底层功能与系统之间的配合，这些要求不高的任务可以从主ARM处理器上脱离出来，从而提升整体的性能。  
换句话说，系统中存在的 ARM 处理器并不会妨碍软处理器的使用，甚至很多应用能因采用了此种类型的处理模式（软、硬处理器并存）而受益。
![](1.png)
***

**重要的是**，Zynq 的处理器系统里并非只有 ARM 处理器，还有一组相关的处理资源，形成了一个应用处理器单元 （Application Processing Unit，APU） ，另外还有扩展外设接口、cache 存储器、存储器接口、互联接口和时钟发生电路。图2.2 所示是 PS 部分架构框图，其中高亮的部分就是 APU。
![image.png](2.png)

### 2.1.2 应用处理器单元 （APU）
***
图 2.3 所示是 APU 的简化框图。APU 主要是由两个 ARM 处理核组成的，每个都关联了一些可计算的单元：一个 NEONTM 媒体处理引擎（Media Processing Engine，MPE）和浮点单元 （Floating Point Unit，FPU）；一个内存管理单元 （MemoryManagement Unit，MMU）；和一个一级 cache 存储器（分为指令和数据两个部分）。APU 里还有一个二级 cache 存储器，再往下还有片上存储器 （On Chip Memory，OCM）。最后，由一个一致性控制单元 （Snoop Control Unit，SCU）在 ARM 核和二级 cache 及 OCM 存储器之间形成了桥连接，这个单元还部分负责与 PL 对接，图中没有标出这个接口。
![](3.png)

### 2.1.3 关于 ARM 模式

### 2.1.3. 处理器系统外部接口

## 2.2. 可编程逻辑
***

### 2.2.1. 逻辑部分
图 2.5 描绘了 Zynq 芯片的 PL 部分，其中几个功能被高亮了出来。PL 主要是由通用 FPGA 逻辑部分组成的，这个 FPGA 是由逻辑片和可配置逻辑块 （Configurable Logic Block，CLB）组成的，另外还有用于接口的输入 / 输出块 （Input/ OutputBlock，IOB）（注意这些都是 Xilinx 专有的术语）。
![PL](PL.png)
* **可配置逻辑块 (CLB)** — CLB 是逻辑单元的小规模、普通编组，在 PL 中排列为一个二维阵列，通过可编程互联连接到其他类似的资源。每个 CLB 里包含两个逻辑片，并且紧邻一个开关矩阵，如图 2.6 所示。
* **片 （Slice）** — CLB 里的一个子单元，里面有实现组合和时序逻辑电路的资源。如图 2.6 所示，Zynq 的片是由 4 个查找表、8 个触发器和其他一些逻辑所组成的。
* **查找表 （Lookup Table，LUT）** — 一个灵活的资源，可以实现 （一）至多 6个输入的逻辑函数； （二）一小片只读存储器 （ROM）； （三）一小片随机访问存储器 （RAM）；或 （四）一个移位寄存器。LUT 可以按需组合起来形成更大的逻辑函数、存储器或移位寄存器。
*  **触发器（Flip-flop，FF）** — 一个实现 1 位寄存的时序电路，带有复位功能。FF 的一种用处是实现锁存。
* **开关矩阵 （Switch Matrix)** — 每个 CLB 旁都有一个开关矩阵，实现灵活的布线功能来（一）连接 CLB 内的单元；或（二）把一个 CLB 与 PL 内的其他资源连接起来。
* **进位逻辑 （Carry Logic）** — 算术电路需要在相邻的片之间传递信号，这就是通过进位逻辑来实现的。进位逻辑把布线和复用器组成链条来连接一个垂直列上的片。
* **输入 / 输出块 （Input/Output Blocks，IOB）** — IOB 实现了 PL 逻辑资源之间的对接，并且提供物理设备 “ 焊盘 ” 来连接外部电路。每个 IOB 可以处理一位的输入或输出信号。IOB 通常位于芯片的周边。
![](CLB.png)

### 2.2.2. 特殊资源：DSP48E1 和块 RAM
***
除了通用的部分，还有两个特殊用途的部件：满足**密集存储需要的块 RAM 和用于高速算术的 DSP48E1 片**
* 使用块 RAM 就意味着能在芯片内优化的专用存储单元内，用很小的物理空间储存大量的数据。   <==>分布式 RAM （Distributed RAM）
* DSP48E1是专门用于实现对长字长信号的高速算术运算的逻辑片。
    * DSP48E1 用了复用电路来灵活使用寄存器，并且能支持计算的动态变更（就是说功能可以按需要逐个周期地改变） 。多种计算是可能的，包括一个、两个或所有的算术运算符，这些可通过 OPMODE 输入来选择，这个输入会配置内部的复用器 （没有完整显示在图中），并决定所实现的算术功能。
    * 后加法器有一个额外的功能，可以用作**逻辑单元**。
    * 如果需要的话，也可以**组合多个 DSP48E1 来做扩展**。高频率、低功耗，对于实现计算密集型算术电路是很有吸引力的。
    * 适合**信号处理及其他各种应用**
![](2.8.png)
![](4.png)

### 2.2.3. 通用输入 / 输出
* Zynq 上的通用输入 / 输出功能（IOB）合起来被称作 SelectIO 资源，它们被组织成 50 个 IOB 一组。每个 IOB 有一个焊盘，是与外部世界连接来做单个信号的输入或输出的。  
* I/O 组被分类为高性能 （High Performance，HP）或高范围 （High Range，HR） ，支持各种 I/O 标准和电压。  
* 每个 IOB 还包含一个 IOSERDES 资源，可以做并行和串行数据的可编程转换（串行化和反串行化） ，数据可以是 2 位到 8 位的。

### 2.2.4. 通信接口
更准确地说，Zynq 芯片里含有嵌入在逻辑部分里的 GTX 收发器和高速通信接口块

### 2.2.5. 其他可编程逻辑扩展接口
* 模拟 - 数字转换
* 时钟 - PL 接收来自 PS 的四个独立的时钟输入，另外还能产生和分发它自己的与 PS 无关的时钟信号 
* 编程与调试

## 2.3. 处理器系统与可编程逻辑的接口
***
Zynq 的表现不仅仅依赖于它的两个组成部分 PS 和 PL 的特性，还在于能把两者协同起来形成完整、集成的系统的能力。  
本节讨论 PS 和 PL 之间的连接，并探讨如何使用这些连接。

### 2.3.1. AXI 标准

### 2.3.2. AXI 互联和接口
在 PS 和 PL 之间的主要连接是通过一组 9 个 AXI 接口，每个接口有多个通道组成。这些形成了 PS 内部的互联以及与 PL 的连接，如图 2.9 所示。这里，有必要定义两个重要的术语：
* 互联（Interconnect） — 互联实际上是一个开关，管理并直接传递所连接的AXI 接口之间的通信。在 PS 内有几个互联，其中有些还直接连接到 PL （如图2.9） ，而另一些是只用于内部连接的。这些互联之间的连接也是用 AXI 接口所构成的。
* 接口 （Interface） — 用于在系统内的主机和从机之间传递数据、地址和握手信号的点对点连接。
![](2.9.png)

### 2.3.3. EMIO 接口

### 2.3.4. 其他 PL-PS 信号

## 2.4. 安全
***
随着定制 IP 资源的开发和市场的发展趋势，使用具有对其内部软件和硬件 IP 提供安全防护能力的芯片，成为像航空、汽车、广播、工业和有线 / 无线网络及通信这样的市场中日益增长的需求

### 2.4.1. 安全引导
### 2.4.2. 硬件支持
### 2.4.3. 运行时刻安全


## 2.5. 本章回顾
***
本章概述了 Zynq 芯片的总体架构，并详述了它的两个组成部分：PS 和 PL，以及介于这两部分之间的接口资源。

处理器系统包含了一个双核 ARM Cortex-A9 处理器，带有合成在一个 APU 单元
里的 SIMD 和浮点运算的扩展，另外还有存储器资源。着重介绍了 APU 的能力及其与
处理器系统中其他部分的接口。同时也解释了 PS 整体的内部结构，及其与可编程逻
辑的连接。

概述了可编程逻辑部分的结构和资源， 包括逻辑部分、块 RAM 和 DSP48E1 资源，
及接口资源，并说明了 PL 和 PS 之间用 AXI 构成的接口的重要问题。最后，还比较
了 Zynq-7000 系列的不同芯片。